In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinBiolinum_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.2)

In [ ]:
data = pd.concat([pd.read_csv(data, names=[
    "Mode", "Variant", "Number of Processors", "Latency"
]) for data in glob('saved/mutex/*.txt')], ignore_index=True)
# data

# Latency distribution of every processor requesting once

Notice the significant larger spans of replicated. All requesting are concurrent while there is a sequential causality chain of processor#1 requested-processor#1 releasing-processor#2 requested-processor#2 releasing-processor#3 requested-... since the first processor is granted with the resource. The large spans are clear indicator of replicated being a more serial system.

In [ ]:
# plt.figure(figsize=(12, 3))
f = sns.boxplot(data=data[data["Mode"] == "All"],
    x="Number of Processors", y="Latency", hue="Variant", 
    hue_order=["Untrusted", "Replicated", "Quorum", "NitroEnclaves"], dodge=False)
f.set(ylabel="Latency (s)")
f.set(ylim=(0, 28))
f.figure.savefig("mutex-all.pdf", bbox_inches='tight')

# Latency of only one processor requesting

In [ ]:
f = sns.lineplot(data=data[data["Mode"] == "One"],
    x="Number of Processors", y="Latency", hue="Variant", 
    hue_order=["Untrusted", "Replicated", "Quorum", "NitroEnclaves"],
    style="Variant", markers=["o"])
f.set(ylim=(0, 2.0))
f.set(ylabel="Latency (s)")
f.figure.savefig("mutex-one.pdf", bbox_inches='tight')

# The measurement of system concurrency

The ratio of *latency of all processors are granted once*/*latency of one processor granted*. The more flat the line is, the more requests progress concurrently.

Notice that the Quorum/NitroEnclaves solutions have $N^2$ per-processor steps for collecting acquisition proof. Replicated also have $N^2$ per-processor steps for replication (though reduced by batching).

In [ ]:
data_all_max_latency = data[data['Mode'] == 'All'].groupby(['Variant', 'Number of Processors']).max()
data_one_max_latency = data[data['Mode'] == 'One'].groupby(['Variant', 'Number of Processors']).max()
data_ratio = data_all_max_latency.merge(data_one_max_latency, how='inner', on=['Variant', 'Number of Processors'])
data_ratio['Ratio'] = data_ratio['Latency_x'] / data_ratio['Latency_y']
# data_ratio

In [ ]:
f = sns.lineplot(data=data_ratio,
    x="Number of Processors", y="Ratio", hue="Variant", 
    hue_order=["Untrusted", "Replicated", "Quorum", "NitroEnclaves"],
    style="Variant", markers=["o"])